# Regional Fields

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

Regional fields are fields in a `Material` where the values are assigned to points by region in the material. Common uses of regional fields include assigning properties to different phases or orientations to different grains. Regional fields can be accessed exactly like regular fields, but there are also some additional utilities that are available.

First, create a `Material` with points assigned to different phases.

In [ ]:
import numpy as np

from materialite import Material, Box
from materialite.tensor import Orientation

material = Material(dimensions=[4, 4, 4]).create_uniform_field("phase", 1)
box = Box(max_corner=[1, np.inf, np.inf])
material = material.insert_feature(box, fields={"phase": 0})
material.plot("phase")

Add a regional field: values of the `youngs_modulus` field will be assigned to points in the `Material` based on the value of `phase`. The arguments of the `create_regional_field` method are the label of the field that contains the region IDs (`"phase"` in this case) and a dictionary or Pandas `DataFrame` containing the unique region IDs and the corresponding values of the regional field(s).

In [ ]:
regional_field = {"phase": [0, 1], "youngs_modulus": [10, 1000]}
material = material.create_regional_fields(
    region_label="phase", regional_fields=regional_field
)

Use `get_fields` to check that values of the regional field are assigned correctly. Points with `phase=0` and `phase=1` should have `youngs_modulus=10` and `youngs_modulus=1000`, respectively.

In [ ]:
material.get_fields()

We can also `extract` or `plot` the pointwise values of the `youngs_modulus` field.

In [ ]:
material.extract("youngs_modulus")

In [ ]:
material.plot("youngs_modulus")

You can add to already-existing regional fields. The regional fields can also be Materialite `Tensor`s, just like with regular fields.

In [ ]:
orientations = Orientation.from_euler_angles([[0, 0, 0], [np.pi / 4, 0, 0]])
regional_field2 = {
    "phase": [1, 0],
    "orientation": orientations,
}
material = material.create_regional_fields("phase", regional_field2)

Note that points with `phase=1` should have all zero Euler angles, and points with phase=0 should not.

In [ ]:
material.get_fields()

You can use `extract_regional_field` to get a `DataFrame` containing all the regional fields corresponding to a particular region label.

In [ ]:
material.extract_regional_field(region_label="phase")

We can also use `extract_regional_field` to extract values of one regional field.

In [ ]:
material.extract_regional_field(region_label="phase", field_label="orientation")

If you need to do any indexing, you can use `get_region_indices` to extract the indices of `material.fields` corresponding to each region in the regional field. This method returns a dictionary whose keys are the region IDs and whose values are the associated indices.

In [ ]:
indices = material.get_region_indices("phase")
print(f"keys: {indices.keys()}")
print(f"values: {indices.values()}")

Regional fields can be deleted

In [ ]:
material.remove_field(field_label="orientation", in_regional_field="phase").get_fields()

Deleting any field that has corresponding regional fields will delete the regional fields as well.

In [ ]:
material.remove_field(field_label="phase").get_fields()

Materialite raises an error in 4 cases
1. The `Material` does not have a field with the provided `region_label`

In [ ]:
try:
    bad_regional_field = {"grain": [1, 2], "value": [3, 4]}
    material.create_regional_fields("grain", bad_regional_field)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

2. The `Material` already has a field with the same label as a new field provided in the regional fields.

In [ ]:
try:
    bad_regional_field = {"phase": [0, 1], "x": [3, 4]}
    material.create_regional_fields("phase", bad_regional_field)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

3. The regional field has duplicated values in the `region_label` field (in this case, `phase`)

In [ ]:
try:
    bad_regional_field = {"phase": [0, 1, 0], "value": [3, 4, 5]}


    material.create_regional_fields("phase", bad_regional_field)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

4. The `region_label` field (in this case, `phase`) in the `Material` has values that are not contained in the region field.

In [ ]:
try:
    bad_region_field = {"x": [0, 2, 3, 4], "value": [3, 4, 5, 6]}


    material.create_regional_fields("x", bad_region_field)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

In [ ]:
try:
    bad_region_field = {"phase": [0, 2], "value": [3, 4]}


    material.create_regional_fields("phase", bad_region_field)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

You will also get an error if two regional fields define the same field, but only on the next call to `get_fields()`

In [ ]:
try:
    x_regional_fields = {"x": [0, 1, 2, 3], "youngs_modulus": [10, 100, 1000, 10000]}
    material.create_regional_fields("x", regional_fields=x_regional_fields).get_fields()
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")